# AntiPhish-LLM

This is AntiPhish-LLM, a tool that leverages OpenAI's GPT-4 for the automatic classification of phishing emails and generation of warning messages.

This tool was used in the study "Can LLMs help protect users from phishing attacks? An exploratory study", submitted for the CHI'24 conference, Late-Breaking Work track.

# Setup

Firstly, we install the needed python libraries.

In [ ]:
!pip install scipy
!pip install tenacity
!pip install cohere
!pip install tiktoken
!pip install termcolor
!pip install openai
!pip install requests
!pip install evals
!pip install beautifulsoup4
!pip install dnspython

We then define some utility functions.

In [ ]:
import json
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4-1106-preview" # "gpt-3.5-turbo-0613"

# Replace with your OpenAI API key
api_key = ''

# Set the API key
openai.api_key = api_key

# Utility functions definition

@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model="gpt-3.5-turbo", temperature=0, n_choices=1):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "temperature": temperature, "messages": messages, "n": n_choices}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


# Email Preprocessing and URL info gathering

We define hereafter the function for preprocessing the email.

We pre-process the email following the approach used in [K. Misra and J. T. Rayz, "LMs go Phishing: Adapting Pre-trained Language Models to Detect Phishing Emails," 2022 IEEE/WIC/ACM International Joint Conference on Web Intelligence and Intelligent Agent Technology (WI-IAT), Niagara Falls, ON, Canada, 2022, pp. 135-142, doi: 10.1109/WI-IAT55865.2022.00028.](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10101955)

In [ ]:
import re
import urllib.parse
from bs4 import BeautifulSoup
import quopri
import email

# Emails pre-processing

def get_fullhostname(url):
    # get the domain name from the URL (without the path)
    re_match = re.search(r"^(\w+:\/\/)?(?:[^@\/\n]+@)?((?:www\.)?[^:\/?\n]+)\:?(\d*)(\/[^?]*)?(\?.*)?", url, re.I)
    if (re_match != None):
      re_match = re_match.groups()
      full_hostname = re_match[0] + re_match[1]
    return full_hostname

def preprocess_email(email_content):
  # Parse the email content
  parser = email.parser.BytesParser()
  email_message = parser.parsebytes(email_content)

  ## Extract the subject
  subject = email_message['subject'] or "NO SUBJECT"

  ## Extract the email headers
  headers = email_message.items()
  # Convert the headers to a string
  header_string = "\n".join([f"{key}: {value}" for key, value in headers])

  ## Extract the email body
  body = ""
  if email_message.is_multipart():
      # If the email has multiple parts (e.g., text and HTML), we iterate through them
      for part in email_message.walk():
          content_type = part.get_content_type()
          if content_type == 'text/plain' or content_type == 'text/html':
              body += part.get_payload(decode=True).decode()
  else:
      # If the email is not multipart, it's a single plain text message
      body = email_message.get_payload(decode=True).decode()

  # Body pre-processing
  urls_list = []
  if body.find("Content-Transfer-Encoding: quoted-printable") != -1:
    print ("Quoted-printable content")
    decoded_bytes_object = quopri.decodestring(body)
    body = decoded_bytes_object.decode("utf-8", errors="ignore")  # TODO: get the right charset
  soup = BeautifulSoup(body, 'html.parser')

  for a_tag in soup.find_all(re.compile('a|img|div', re.I)):  # we try to find URLS in the href attribute of a, img, and div tags
    href = a_tag.get("href")
    if href != None:
      if href.startswith('tel') or href.startswith('sms'):
        metatag = "PHONE"
        href = href.replace(r'(tel|sms):', '')
      elif href.startswith('mailto'):
        metatag = "EMAIL"
        href = href.replace('mailto:', '')
      else:
        metatag = "URL"
        urls_list.append(href)
      visible_string = a_tag.string or ""
      a_tag.replace_with(f'[{metatag} HREF="{href}"] {visible_string} [/{metatag}]')

    body = soup.get_text()
    # get the initial part of the URL only [protocol+FQDN(fully qualified domain name)] \g<1> = protocol (+ www.), \g<2> = FQDN
    body = re.sub(r"(https?:\/\/|www\.)([-a-zA-Z0-9@:%._\-\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6})\b[-a-zA-Z0-9()@:%_+.~#?&\/=\-]*", r"\g<1>\g<2>", body)

    body = re.sub(r" {2,}", " ", body)  # remove duplicate blanks
    body = re.sub(r"\n{2,}", "\n", body) # remove duplicate \n chars
    #body = urllib.parse.parseqsl(body)
    #body = body.replace(r'=[0-9A-F]{2}', '')

  return {
      "headers": headers,
      "subject" : subject,
      "body" : body,
      "urls" : urls_list
  }

## URL info

We define the functions to collect online information about the URL(s) in the email.

In [ ]:
import requests
import base64
import dns.resolver
import socket

vt_api_key = '' # VirusTotal API key - https://www.virustotal.com/gui/home/upload
blacklist_api_key = ''  # Blacklist Checker API key - https://blacklistchecker.com/
dns_api_key = '' # BigDataCloud API key - https://www.bigdatacloud.com/

def get_ip_addr(url):
    # remove protocol from URL (if exists)
    match_result = re.match(r"(?:.*\:\/\/)([^\/]*)", url)
    if match_result is not None:
      match_groups = match_result.groups()
      if len(match_groups) > 0:
        url = match_groups[0]
    # dns lookup
    a = dns.resolver.resolve_name(url, family=socket.AF_INET)
    addrs = a.addresses()
    for addr in addrs:
      return addr

## VirusTotal
def get_virustotal_data(url):
    api_base_url = 'https://www.virustotal.com/api/v3/urls/'
    # Headers with the API key
    headers = {
        'x-apikey': vt_api_key,
    }
    base_64_url = base64.b64encode(url.encode('ascii')) # encode the url in base64 bytes
    base_64_url = base_64_url.decode("ascii")  # get the base64 string
    base_64_url = base_64_url.rstrip('=')  # remove the trailing padding chars '='
    request_url = api_base_url + base_64_url

    response = requests.get(request_url, headers=headers) # Make the HTTP GET request

    # Check for a successful response (HTTP status code 200)
    if response.status_code == 200:
        # Access the JSON response
        result = response.json()
    else:
        print(f"VirusTotal Request failed with status code: {response.status_code}")
    vt_data = result['data']['attributes']['last_analysis_stats']  # contains the votes for the scan

    response.close() # Close the response
    return vt_data


## Blackist Checker API
def get_blacklists_data(url):
    api_base_url = "https://api.blacklistchecker.com/"
    headers = {
        'authorization': "Basic username" + blacklist_api_key
    }
    request_url = api_base_url + "check/" + url
    response = requests.get(request_url, headers=headers) # Make the HTTP GET request

    # Check for a successful response (HTTP status code 200)
    if response.status_code == 200:
        # Access the JSON response
        result = response.json()
    else:
        print(f"BlacklistChecker Request failed with status code: {response.status_code}")
        result = {"detections": 0}
    n_blacklists_found = result["detections"]  # The detections field simply carries the number of blacklists in which the domain appeared

    response.close() # Close the response
    return n_blacklists_found


## BigDataCloud API (get location of IP address)
def get_dns_info(url):
    api_base_url = "https://api.bigdatacloud.net/data/country-by-ip"
    ip_addr = get_ip_addr(url)

    get_params = {
        "ip" : ip_addr,
        "key" : dns_api_key
    }
    request_url = api_base_url
    response = requests.get(request_url, params=get_params) # Make the HTTP GET request

    # Check for a successful response (HTTP status code 200)
    if response.status_code == 200:
        # Access the JSON response
        result = response.json()
        # print (result)
    else:
        print(f"BigDataCloud Request failed with status code: {response.status_code}")
        result = {"country": None}
    country = result["country"]
    if country != None:
       # countryName = country["name"]  regionName = country["wbRegion"]["value"]
       countryID = country['isoAlpha3']  # country['isoName']
    else:
      countryID = "unknown"
      # countryName = "unknown"  regionName = "unknown"
    response.close() # Close the response
    return countryID # countryName, regionName


def get_url_info(url, string_out=False):
    url = get_fullhostname(url_to_analyze) # gets the full host name (protocol + fqdn), without the URL path
    vt_data = get_virustotal_data(url)
    n_blacklists_found = get_blacklists_data(url)
    domain_location = get_dns_info(url)

    url_info = {
        'Server location' : domain_location,
        'VirusTotal scan' : vt_data,
        'Blacklists' : n_blacklists_found
    }
    if string_out:
      return str(url_info)
    else:
      return url_info


Let's finally open an email and preprocess it.
For now, we gather only URL info for the first URL in the email

In [ ]:
# Open and preprocess an email
email_filename = "email_name.eml" # ENTER THE EMAIL FILE NAME HERE
with open(email_filename, "rb") as email_byes:
  mail = email_byes.read()
  mail = preprocess_email(mail)
  # Print or use the extracted subject, header, and body as needed
  """print("Subject:", mail["subject"])
  print("Headers:")
  print(mail["headers"])
  print("Body:")
  print(mail["body"])
  print("URLS:")
  print(mail["urls"])"""
# Gather additional information about URLs in the email
if len(mail["urls"]) > 0:
  # Call remote API to gather online URL information
  url_to_analyze = mail["urls"][0]  # for now we take the first URL
  url_info = get_url_info(url_to_analyze);

# Classify the email with GPT-4

Once we have our preprocessed email and URL information, we call GPT-4 for the classification.

We already did manual prompt engineering and came up with the following prompt:


```
You are a cybersecurity expert that has the goal to detect if an email is legitimate or phishing and help the user understand the decision.
The email is accompanied by information of the URLs in the email like: server location, in how many blacklists the site was found, VirusTotal scans reporting the number of scanners that detected the URL as harmless, undetected, suspicious, malicious.
Your goal is to output a percentage indicating the probability of the email being phishing (0%=email is surely legitimate, 100%=email is surely phishing). Specifically, you should consider if and how many persuasion principles are applied by the alliged attacker; for each principle, you should report the part of the email that makes you think that persuasion principle is being applied; also add a brief rationale for each.
Moreover, in the cases where the email is suspicious, you have to report 3 to 5 features that could indicate the danger in the email understandable by users with no cybersecurity or computers expertise
Desired format:
Label: [phishing/legit]
Phishing Probability: [0-100%]
Persuasion Principles found: <bullet list of persuation principles + specific sentences + rationale>
Explanation: <bullet list of 3-5 features explained>
```

Let's modify it a bit and set the code for the API call using Chat Completions ([source](https://platform.openai.com/docs/guides/gpt/chat-completions-api)).

## Generating a warning message with explanation

Now we have a classification for an email that also carries a lot of
information regarding the rationale for the classification and the persuasion principles that might have been used.

Nonetheless, we want to have an explanation message that would be easy to understand also by lay users. Therefore, we create another prompt to further refine this longer explanation in an effective warning message.

We created this prompt:



```
Now take the most relevant feature among the ones in your explanations and construct a brief explanation message (max 50 words) directed to naive users (with no knowledge of cybersecurity) that will follow this structure:`
1. description of the most relevant phishing feature
2. explanation of the hazard
3. consequences of a successful phishing attack
For example, an explanation that explains that the top-level domain in one of the email's URL is mispositioned would be:
"In the URL present in the email the top-level domain is in an abnormal position. This could indicate that the URL leads to a fake website. Such websites might steal your personal information”.
Another example of explanation about the domain of a website being suspiciously young would be:
"The URL in the email leads to a website created N days ago. Young websites are famous for criminal activity. There is a potential risk if you proceed."
Another example of explaining that the email is suspicious because of too many special characters in its body would be:
"Many special characters have been detected in the email. Malicious people use them to disguise text and deceive you. Your data could be stolen."


Desired format:
[description of the feature]. [hazard explanation]. [consequences of a successful attack].
```


In [ ]:
SEED = 42
MODEL =  "gpt-4-1106-preview" # "gpt-3.5-turbo-1106"
TEMPERATURE = 0

def classify_email(email_input, feature_to_explain=None, url_info=None, explanations_min=3, explanations_max=6, model=MODEL):
    # Initial Prompt
    messages = [{"role": "system", "content": f'''You are a cybersecurity and human-computer interaction expert that has the goal to detect
        if an email is legitimate or phishing and help the user understand why a specific email is dangerous (or genuine), in order
        to make more informed decisions.
        The user will submit the email (headers + subject + body) optionally accompanied by information of the URLs in the email as:
        - server location;
        - VirusTotal scans reporting the number of scanners that detected the URL as harmless, undetected, suspicious, malicious;
        - number of blacklists in which the linked domain was found.

        Your goal is to output a JSON object containing:
        - The classification result (label).
        - The probability in percentage of the email being phishing (0%=email is surely legitimate, 100%=email is surely phishing) (phishing_probability).
        - A list of persuasion principles that were applied by the alliged attacker (if any); each persuasion principle should be an object containing:
            the persuasion principle name (authority, scarcity, etc.),
            the part of the email that makes you say that persuasion principle is being applied;
            a brief rationale for each principle.
        - A list of {explanations_min} to {explanations_max} features that could indicate the danger (or legitimacy) of the email; the explanations must be understandable by users with no cybersecurity or computers expertise.


        Desired format:
        label: <phishing/legit>
        phishing_probability: <0-100%>
        persuasion_principles: [array of persuation principles, each having: {{name, specific sentences, rationale}} ]
        explanation: [array of {explanations_min}-{explanations_max} features explained]'''
      }]
    # User input (email)
    headers = str(email_input["headers"])
    subject = email_input["subject"]
    body = email_input["body"]
    email_prompt = f'''Email:
          """
          [HEADERS]
            {headers}
          [\HEADERS]
          [SUBJECT] {subject} [\SUBJECT]
          [BODY]
          {body}
          [\BODY]
          """
          '''
    # Add the url_info if it exists
    if url_info is not None:
      email_prompt += f"""

          ######

          URL Information:
          {str(url_info)}"""

    messages.append({"role": "user", "content": email_prompt})
    # Get the classification response
    response = openai.chat.completions.create(
      model=MODEL,
      seed=SEED,
      temperature=TEMPERATURE,
      messages=messages,
      response_format = {"type": "json_object"}
    )
    classification_response = response.choices[0].message.content

    messages.append({"role": "assistant", "content": f"{classification_response}"}) # attach the response string for the second prompt

    # Try getting the JSON object from the response
    try:
      classification_response = json.loads(classification_response)
    except:
      print ("Invalid JSON format in the response")
      return classification_response, ""


    if "label" in classification_response:
      predicted_label = classification_response['label']
      if predicted_label == "legit":
        # If the classification == legit, then exit the function
        return classification_response, "The email is genuine"
      else:
        # Otherwise, we ask GPT to produce the warning message
        if feature_to_explain == None:
          # Automatically take the most relevant feature
          messages.append(
              {"role": "user", "content": """
              Now take the most relevant feature among the ones in your explanations and construct a brief explanation message (max 50 words) directed to naive users (with no knowledge of cybersecurity) that will follow this structure:`
              1. description of the most relevant phishing feature
              2. explanation of the hazard
              3. consequences of a successful phishing attack
              For example, a message that explains that a URL in the email (PHISHING_URL) is imitating another legitimate one (SAFE_URL), would be:
              "The target URL [PHISHING_URL] is an imitation of the original one, [SAFE_URL]. This site might be intended to take you to a different place. You might be disclosing private information.”.
              Another example of explanation about the domain of a website being suspiciously young would be:
              "This website is very young (created [N] days ago). Fraudulent websites have a similar age. There is a potential risk of being cheated if you proceed."
             Another example of explaining that the email is suspicious because a domain linked in the email is hosted in a country with bad reputation would be:
              "The host of the target website is in [COUNTRY], which is where most attacks originate. Sharing your private information here is risky."

              Desired format:
              [description of the feature]. [hazard explanation]. [consequences of a successful attack].
              """}
            )
        else:
          # Be primed about the feature to explain
          messages.append(
              {"role": "user", "content": f"""
              Consider that the previous email is suspicious because {feature_to_explain["description"]}: construct a brief explanation message (max 50 words) directed to naive users (with no knowledge of cybersecurity) that will follow this structure:`
              1. description of the feature (in this case {feature_to_explain["name"]})
              2. explanation of the hazard
              3. consequences of a successful phishing attack
              For example, an explanation that explains that the top-level domain in one of the email's URL is mispositioned would be:
              "In the URL present in the email the top-level domain is in an abnormal position. This could
              indicate that the URL leads to a fake website. Such websites might steal your personal
              information".
              Another example of explanation about the domain of a website being suspiciously young would be:
              "The URL in the email leads to a website created N days ago. Young websites are famous for criminal activity. There is a potential risk if you proceed."
              Another example of explaining that the email is suspicious because of too many special characters in its body would be:
              "Many special characters have been detected in the email. Malicious people use them to disguise text and deceive you. Your data could be stolen."

              Desired format:
              [description of the feature]. [hazard explanation]. [consequences of a successful attack].
              """}
            )


        response_2 = openai.chat.completions.create(
          model=MODEL,
          seed=SEED,
          temperature=TEMPERATURE,
          messages=messages
        )
        explanation_response = response_2.choices[0].message.content
      return classification_response, explanation_response
    else: # Error: response in wrong format
      print ("The response does not contain the predicted label (phishing/non-phishing)")
      return classification_response, ""


Let's generate an email classification + explanation:

In [ ]:
# Call GPT-3.5-turbo for email phishing classification (automatic feature detection)
classification_response_3dot5, warning_msg_3dot5 = classify_email(mail, url_info, model="gpt-3.5-turbo-1106") #=MODEL)

print (classification_response_3dot5)
print (warning_msg_3dot5)

## Generate explanations for specific email by priming on the feature to explain

In [ ]:
# Young domain
# Open and preprocess the email
email_filename = "phishing_young_domain.eml"
with open(email_filename, "rb") as email_byes:
  mail = email_byes.read()
  mail = preprocess_email(mail)
# Call GPT for email phishing classification (priming the model about the feature to explain)
feature_to_explain = {
    "name" : 'Young domain',
    "description" : 'it contains an URL that leads to a domain that is very new'
}
classification_response, warning_msg = classify_email(mail, feature_to_explain=feature_to_explain, url_info=None, model=MODEL)

print (classification_response)
print (warning_msg)

{'label': 'phishing', 'phishing_probability': '95%', 'persuasion_principles': [{'name': 'Authority', 'specific_sentences': 'BRT Pacchetto in attesa', 'rationale': 'The email impersonates an authoritative courier service to create a sense of legitimacy and urgency.'}, {'name': 'Scarcity', 'specific_sentences': 'Hai (1) pacco in attesa di consegna.', 'rationale': 'The message suggests that there is a limited time to act on the delivery, which can pressure the recipient into taking hasty actions without proper verification.'}, {'name': 'Consistency', 'specific_sentences': 'Pianifica la tua consegna e iscriviti ai nostri avvisi di calendario per evitare che ciò accada di nuovo!!', 'rationale': 'The email prompts the user to take immediate action to resolve an issue, playing on the desire to maintain consistency and follow through with commitments, such as receiving a package.'}], 'explanation': ["The sender's email address ('jessicahwhite47879@gmail.com') does not match the official email 

In [ ]:
# IP Address
# Open and preprocess the email
email_filename = "phishing_IP.eml"
with open(email_filename, "rb") as email_byes:
  mail = email_byes.read()
  mail = preprocess_email(mail)
# Call GPT for email phishing classification (priming the model about the feature to explain)
feature_to_explain = {
    "name" : "URL is IP address",
    "description" : "it contains an URL that is an IP address"
}
classification_response, warning_msg = classify_email(mail, feature_to_explain=feature_to_explain, url_info=None, model=MODEL)

print (classification_response)
print (warning_msg)

In [ ]:
# TLD mispositioned
# Open and preprocess the email
email_filename = "phishing_TLD_mispositioned.eml"
with open(email_filename, "rb") as email_byes:
  mail = email_byes.read()
  mail = preprocess_email(mail)
# Call GPT for email phishing classification (priming the model about the feature to explain)
feature_to_explain = {
    "name" : "Top-Level Domain mispositioned",
    "description" : "it contains an URL with a top-level domain (.com) found as a subdomain"
}
classification_response, warning_msg = classify_email(mail, feature_to_explain=feature_to_explain, url_info=None, model=MODEL)

print (classification_response)
print (warning_msg)

In [ ]:
# Link mismatch
# Open and preprocess the email
email_filename = "phishing_link_mismatch.eml"
with open(email_filename, "rb") as email_byes:
  mail = email_byes.read()
  mail = preprocess_email(mail)
# Call GPT for email phishing classification (priming the model about the feature to explain)
feature_to_explain = {
    "name" : "Link mismatch",
    "description" : "it contains a displayed link that is different from the actual pointed URL"
}
classification_response, warning_msg = classify_email(mail, feature_to_explain=feature_to_explain, url_info=None, model=MODEL)

print (classification_response)
print (warning_msg)

In [ ]:
# Call GPT-4-turbo for email phishing classification
classification_response_4, warning_msg_4 = classify_email(mail, url_info, model="gpt-4-1106-preview")

print (classification_response_4)
print (warning_msg_4)

# Output Examples

### Using GPT-3.5:
Example of a wrong classification (false positive) - we must acknowledge that it classified it correctly the second time we ran the prompt:
```
{
  'label': 'phishing',
  'probability': '95%',
  'persuasion_principles':
  [
    'Authority: The email uses the name of ngrok founder and CEO Alan Shreve to establish credibility and persuade the recipient to click on the links.',
    "Scarcity: The email mentions 'Pay-as-you-go pricing' and 'Secure your ngrok account with MFA' to create a sense of urgency and persuade the recipient to take immediate action.",
    'Social Proof: The email includes links to GitHub, LinkedIn, Twitter, and YouTube to show social validation and persuade the recipient to trust the content.'
  ],
  'explanation':
  [
    'Unsolicited URLs: The email contains multiple unsolicited URLs, which is a common tactic used in phishing emails to lure recipients to malicious websites.',
    "Urgent Call to Action: The email creates a sense of urgency by promoting 'Pay-as-you-go pricing' and 'Secure your ngrok account with MFA', which is a typical strategy used in phishing emails to prompt immediate action.",
    "Unverified Sender: The email claims to be from 'ngrok', but the sender's address is from a suspicious domain '21124867m.ngrok-info.com', indicating a potential impersonation attempt."
  ]
}
```

Warning message:
**The email contains multiple unsolicited URLs, a common tactic used in phishing emails to lure recipients to malicious websites. Clicking on these links could lead to fake websites designed to steal personal and financial information, potentially resulting in identity theft or financial loss.**


### Using GPT-4 (as in the final tool):

Same email, correct classification

```
{'label': 'legit',
 'probability': '0%',
 'persuasion_principles': [],
 'explanation': ["The email passed SPF, DKIM, and DMARC checks, which are email authentication methods that help prevent email spoofing. The headers show 'spf=pass', 'dkim=pass', and 'dmarc=pass', indicating that the email was sent from an authorized server and the content has not been tampered with.",
  "The sender's domain 'ngrok.com' is consistent with the content of the email, which discusses ngrok's services. This alignment between the sender and the content suggests that the email is legitimate.",
  'The email contains multiple URLs, but all of them point to the same domain, which is consistent with legitimate marketing practices where a company promotes its own content and services.',
  'The VirusTotal scan reports that the URL was detected as harmless by 71 scanners and undetected by 18, with no scanners flagging it as suspicious or malicious. This indicates that the URL is safe to visit.',
  'The linked domain was not found on any blacklists, which suggests that it is not associated with any known malicious activities.',
  'The email provides a clear way to unsubscribe or manage preferences, which is a common practice in legitimate marketing emails to comply with anti-spam laws.']}
  ```
  
No warning message - **The email is genuine**
